**Demo for `teneva.core.transformation`**

---

This module contains the functions for orthogonalization and truncation of the TT-tensors.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `full`

For a given TT-tensor (list of TT-cores), calculates the tensor in full format (this function can only be used for relatively small tensors).

In [2]:
n = [10] * 5             # Shape of the tensor      
Y0 = np.random.randn(*n) # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)      # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)          # Print the TT-tensor
Y2 = teneva.full(Y1)     # Compute full tensor from the TT-tensor
abs(np.max(Y2-Y0))       # Compare original tensor and reconstructed tensor

TT-tensor     5D : |10|  |10|   |10|   |10|  |10|
<rank>  =   63.0 :    \10/  \100/  \100/  \10/


1.9539925233402755e-14

## Function `full_matrix`

Export QTT-matrix to the full (numpy) format (this function can only be used for relatively small matrices).

In [3]:
q = 10   # Matrix size factor
n = 2**q # Matrix mode size

# Construct some matrix:
Y0 = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        Y0[i, j] = np.cos(i) * j**2
        
# Construct QTT-matrix / TT-tensor by TT-SVD:
Y1 = teneva.svd_matrix(Y0, e=1.E-6)

# Print the result:
teneva.show(Y1)

# Convert to full matrix:
Y2 = teneva.full_matrix(Y1)

# Compare original matrix and reconstructed matrix
abs(np.max(Y2-Y0))

TT-tensor    10D : |4| |4| |4| |4| |4| |4| |4| |4| |4| |4|
<rank>  =    5.7 :   \4/ \6/ \6/ \6/ \6/ \6/ \6/ \6/ \4/


1.016301712297718e-05

## Function `orthogonalize`

Orthogonalize TT-tensor.

We set the values of parameters and build a random TT-tensor:

In [4]:
d = 5                        # Dimension of the tensor
n = [12, 13, 14, 15, 16]     # Shape of the tensor
r = [1, 2, 3, 4, 5, 1]       # TT-ranks for TT-tensor
Y = teneva.tensor_rand(n, r) # Build random TT-tensor
teneva.show(Y)               # Print the resulting TT-tensor

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


We perform "left" orthogonalization for all TT-cores except the last one:

In [5]:
Z = teneva.orthogonalize(Y, d-1)
teneva.show(Z)

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


We can verify that the values of the orthogonalized tensor have not changed:

In [6]:
# The relative difference ("accuracy"):
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 1.61e-08


And we can make sure that all TT-cores, except the last one, have become orthogonalized (in terms of the TT-format):

In [7]:
for G in Z:
    print(sum([G[:, j, :].T @ G[:, j, :] for j in range(G.shape[1])]))

[[ 1.00000000e+00 -2.77555756e-17]
 [-2.77555756e-17  1.00000000e+00]]
[[1.00000000e+00 6.93889390e-18 8.32667268e-17]
 [6.93889390e-18 1.00000000e+00 7.63278329e-17]
 [8.32667268e-17 7.63278329e-17 1.00000000e+00]]
[[ 1.00000000e+00  3.20923843e-17 -1.04083409e-17  6.93889390e-18]
 [ 3.20923843e-17  1.00000000e+00  2.42861287e-17 -5.89805982e-17]
 [-1.04083409e-17  2.42861287e-17  1.00000000e+00 -2.08166817e-17]
 [ 6.93889390e-18 -5.89805982e-17 -2.08166817e-17  1.00000000e+00]]
[[ 1.00000000e+00  2.77555756e-17 -1.25767452e-17  2.77555756e-17
   2.77555756e-17]
 [ 2.77555756e-17  1.00000000e+00 -6.93889390e-18  0.00000000e+00
  -2.77555756e-17]
 [-1.25767452e-17 -6.93889390e-18  1.00000000e+00  1.38777878e-17
   3.81639165e-17]
 [ 2.77555756e-17  0.00000000e+00  1.38777878e-17  1.00000000e+00
  -2.08166817e-17]
 [ 2.77555756e-17 -2.77555756e-17  3.81639165e-17 -2.08166817e-17
   1.00000000e+00]]
[[86536252.64677295]]


We can also perform "right" orthogonalization for all TT-cores except the first one:

In [8]:
Z = teneva.orthogonalize(Y, 0)

We can verify that the values of the orthogonalized tensor have not changed:

In [9]:
# The relative difference ("accuracy"):
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 0.00e+00


And we can make sure that all TT-cores, except the first one, have become orthogonalized (in terms of the TT-format):

In [10]:
for G in Z:
    print(sum([G[:, j, :] @ G[:, j, :].T for j in range(G.shape[1])]))

[[86536252.64677292]]
[[ 1.00000000e+00 -1.38777878e-17]
 [-1.38777878e-17  1.00000000e+00]]
[[ 1.00000000e+00 -1.03216047e-16 -1.38777878e-17]
 [-1.03216047e-16  1.00000000e+00 -1.04083409e-17]
 [-1.38777878e-17 -1.04083409e-17  1.00000000e+00]]
[[ 1.00000000e+00 -6.59194921e-17  1.56125113e-17  4.16333634e-17]
 [-6.59194921e-17  1.00000000e+00 -1.38777878e-17  1.38777878e-17]
 [ 1.56125113e-17 -1.38777878e-17  1.00000000e+00  4.33680869e-17]
 [ 4.16333634e-17  1.38777878e-17  4.33680869e-17  1.00000000e+00]]
[[ 1.00000000e+00 -1.56125113e-17 -1.11672824e-16  7.63278329e-17
  -4.16333634e-17]
 [-1.56125113e-17  1.00000000e+00  6.39679282e-17  4.16333634e-17
   1.04083409e-16]
 [-1.11672824e-16  6.39679282e-17  1.00000000e+00  4.16333634e-17
  -3.46944695e-18]
 [ 7.63278329e-17  4.16333634e-17  4.16333634e-17  1.00000000e+00
  -5.55111512e-17]
 [-4.16333634e-17  1.04083409e-16 -3.46944695e-18 -5.55111512e-17
   1.00000000e+00]]


We can perform "left" orthogonalization for all TT-cores until i-th and "right" orthogonalization for all TT-cores after i-th:

In [11]:
i = 2
Z = teneva.orthogonalize(Y, i)

for G in Z[:i]:
    print(sum([G[:, j, :].T @ G[:, j, :] for j in range(G.shape[1])]))

G = Z[i]
print('-' * 10 + ' i-th core :')
print(sum([G[:, j, :] @ G[:, j, :].T for j in range(G.shape[1])]))
print('-' * 10)

for G in Z[i+1:]:
    print(sum([G[:, j, :] @ G[:, j, :].T for j in range(G.shape[1])]))

[[ 1.00000000e+00 -2.77555756e-17]
 [-2.77555756e-17  1.00000000e+00]]
[[1.00000000e+00 6.93889390e-18 8.32667268e-17]
 [6.93889390e-18 1.00000000e+00 7.63278329e-17]
 [8.32667268e-17 7.63278329e-17 1.00000000e+00]]
---------- i-th core :
[[23081344.01711273 -5306017.07921848  9384122.28817653]
 [-5306017.07921848 32691745.16366326 -9550300.34658735]
 [ 9384122.28817653 -9550300.34658735 30763163.46599693]]
----------
[[ 1.00000000e+00 -6.59194921e-17  1.56125113e-17  4.16333634e-17]
 [-6.59194921e-17  1.00000000e+00 -1.38777878e-17  1.38777878e-17]
 [ 1.56125113e-17 -1.38777878e-17  1.00000000e+00  4.33680869e-17]
 [ 4.16333634e-17  1.38777878e-17  4.33680869e-17  1.00000000e+00]]
[[ 1.00000000e+00 -1.56125113e-17 -1.11672824e-16  7.63278329e-17
  -4.16333634e-17]
 [-1.56125113e-17  1.00000000e+00  6.39679282e-17  4.16333634e-17
   1.04083409e-16]
 [-1.11672824e-16  6.39679282e-17  1.00000000e+00  4.16333634e-17
  -3.46944695e-18]
 [ 7.63278329e-17  4.16333634e-17  4.16333634e-17  1.0

We can also set a flag "use_stab", in which case a tensor that is 2^p times smaller than the original tensor will be returned (this allows us to preserve the stability of the operation for essentially multidimensional tensors):

In [12]:
Z, p = teneva.orthogonalize(Y, 2, use_stab=True)
Z = teneva.mul(Z, 2**p)

eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 0.00e+00


## Function `orthogonalize_left`

Left-orthogonalization for the i-th TT-core of the given TT-tensor.

We set the values of parameters and build a random TT-tensor:

In [13]:
d = 5                        # Dimension of the tensor
n = [12, 13, 14, 15, 16]     # Shape of the tensor
r = [1, 2, 3, 4, 5, 1]       # TT-ranks for TT-tensor
i = d - 2                    # The TT-core for orthogonalization
Y = teneva.tensor_rand(n, r) # Build random TT-tensor
teneva.show(Y)               # Print the resulting TT-tensor

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


We perform "left" orthogonalization for the i-th TT-core:

In [14]:
Z = teneva.orthogonalize_left(Y, i)
teneva.show(Z)

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


We can verify that the values of the orthogonalized tensor have not changed:

In [15]:
# The relative difference ("accuracy"):
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 0.00e+00


And we can make sure that the updated TT-core have become orthogonalized (in terms of the TT-format):

In [16]:
G = Z[i]
print(sum([G[:, j, :].T @ G[:, j, :] for j in range(G.shape[1])]))

[[ 1.00000000e+00  2.08166817e-17 -3.46944695e-17 -5.89805982e-17
   1.73472348e-17]
 [ 2.08166817e-17  1.00000000e+00 -1.73472348e-17 -2.77555756e-17
  -3.81639165e-17]
 [-3.46944695e-17 -1.73472348e-17  1.00000000e+00  6.93889390e-18
  -2.08166817e-17]
 [-5.89805982e-17 -2.77555756e-17  6.93889390e-18  1.00000000e+00
   5.98479599e-17]
 [ 1.73472348e-17 -3.81639165e-17 -2.08166817e-17  5.98479599e-17
   1.00000000e+00]]


## Function `orthogonalize_right`

Right-orthogonalization for the i-th TT-core of the given TT-tensor.

We set the values of parameters and build a random TT-tensor:

In [17]:
d = 5                        # Dimension of the tensor
n = [12, 13, 14, 15, 16]     # Shape of the tensor
r = [1, 2, 3, 4, 5, 1]       # TT-ranks for TT-tensor
i = d - 2                    # The TT-core for orthogonalization
Y = teneva.tensor_rand(n, r) # Build random TT-tensor
teneva.show(Y)               # Print the resulting TT-tensor

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


We perform "right" orthogonalization for the i-th TT-core:

In [18]:
Z = teneva.orthogonalize_right(Y, i)
teneva.show(Z)

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


We can verify that the values of the orthogonalized tensor have not changed:

In [19]:
# The relative difference ("accuracy"):
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 1.17e-08


And we can make sure that the updated TT-core have become orthogonalized (in terms of the TT-format):

In [20]:
G = Z[i]
print(sum([G[:, j, :] @ G[:, j, :].T for j in range(G.shape[1])]))

[[ 1.00000000e+00  0.00000000e+00 -4.33680869e-17  9.71445147e-17]
 [ 0.00000000e+00  1.00000000e+00 -1.12757026e-17 -6.93889390e-18]
 [-4.33680869e-17 -1.12757026e-17  1.00000000e+00 -4.51028104e-17]
 [ 9.71445147e-17 -6.93889390e-18 -4.51028104e-17  1.00000000e+00]]


## Function `truncate`

Truncate (round) given TT-tensor up to a given accuracy and the rank constraint.

In [21]:
# 10-dim random TT-tensor with TT-rank 3:
Y = teneva.tensor_rand([5]*10, 3)

# Compute Y + Y + Y (the real TT-rank is still 3):
Y = teneva.add(Y, teneva.add(Y, Y))

# Print the resulting TT-tensor
# (note that it has TT-rank 3 + 3 + 3 = 9):
teneva.show(Y)

TT-tensor    10D : |5| |5| |5| |5| |5| |5| |5| |5| |5| |5|
<rank>  =    9.0 :   \9/ \9/ \9/ \9/ \9/ \9/ \9/ \9/ \9/


In [22]:
# Truncate (round) the TT-tensor:
Z = teneva.truncate(Y, e=1.E-2)

# Print the resulting TT-tensor (note that it has TT-rank 3):
teneva.show(Z)

# The relative difference ("accuracy"):
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

TT-tensor    10D : |5| |5| |5| |5| |5| |5| |5| |5| |5| |5|
<rank>  =    3.0 :   \3/ \3/ \3/ \3/ \3/ \3/ \3/ \3/ \3/
Accuracy     : 0.00e+00


We can also specify the desired TT-rank of truncated TT-tensor:

In [23]:
# Truncate (round) the TT-tensor:
Z = teneva.truncate(Y, e=1.E-6, r=3)

# Print the resulting TT-tensor (note that it has TT-rank 3):
teneva.show(Z)

# The relative difference ("accuracy"):
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

TT-tensor    10D : |5| |5| |5| |5| |5| |5| |5| |5| |5| |5|
<rank>  =    3.0 :   \3/ \3/ \3/ \3/ \3/ \3/ \3/ \3/ \3/
Accuracy     : 0.00e+00


If we choose a lower TT-rank value, then precision will be (predictably) lost:

In [24]:
# Truncate (round) the TT-tensor:
Z = teneva.truncate(Y, e=1.E-6, r=2)

# Print the resulting TT-tensor (note that it has TT-rank 2):
teneva.show(Z)

# The relative difference ("accuracy")
eps = teneva.accuracy(Y, Z)

print(f'Accuracy     : {eps:-8.2e}')

TT-tensor    10D : |5| |5| |5| |5| |5| |5| |5| |5| |5| |5|
<rank>  =    2.0 :   \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/ \2/
Accuracy     : 9.82e-01


---